In [7]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
import os 
from time import time
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.datasets import cifar10
import random
import cv2
import numpy as np
from keras.utils import to_categorical
from keras import regularizers

# Basic Guides:
# https://flyyufelix.github.io/2016/10/08/fine-tuning-in-keras-part2.html

batch_size = 128
Classes = 12
N_training = 5000
N_verification = 500

In [2]:
def load_data_plants(source):
    import shutil
    
    source1 = "C:/Data/Workspace/GitHub/DeepLearningProject/LocalContent/Datasets/Resized_180v2/Resized_180/train"
    dest11 =  "C:/Data/Workspace/GitHub/DeepLearningProject/LocalContent/Datasets/Resized_180v2/Resized_180/verf"
    folders = os.listdir(source)

    x_data = []
    
    for f in folders:
        files = os.listdir(source + '/'+ f)
        for i in files:
            x = image.load_img(source + '/'+ f + '/' + i)
            x = img_to_array(x)
            x_data.append(x)
    return x_data

In [3]:
# Configure the loading of data
# Image Generator
# Guied: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,      
    rotation_range=180,
    zoom_range = 0.2,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

# Load data for data augmentation
#x_train = load_data_plants("C:/Data/Workspace/GitHub/DeepLearningProject/LocalContent/Datasets/Resized_180/train")
#x_verf = load_data_plants("C:/Data/Workspace/GitHub/DeepLearningProject/LocalContent/Datasets/Resized_180/verf")

# fit the data augmentation
#train_datagen.fit(x_train)

# fit the data augmentation
#val_datagen.fit(x_verf)
# Clear to save mem
x_train=[]
x_verf=[]
# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '../../LocalContent/Datasets/Resized_180v2/Resized_180/train',  # this is the target directory
       # target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = val_datagen.flow_from_directory(
        '../../LocalContent/Datasets/Resized_180v2/Resized_180/verf',
      #  target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

Found 7350 images belonging to 12 classes.
Found 491 images belonging to 12 classes.


In [4]:
# Configure model
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(4092, activation='relu',kernel_regularizer=regularizers.l2(0.025))(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(Classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [5]:
# Fit the model for the top layers
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# https://stackoverflow.com/questions/45943675/meaning-of-validation-steps-in-keras-sequential-fit-generator-parameter-list
model.fit_generator(
        train_generator,
        steps_per_epoch=N_training//batch_size,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=(N_verification//batch_size),
        max_queue_size=1)

Epoch 1/1
78/78 [==============================] - 168s 2s/step - loss: 13.6305 - val_loss: 3.1272


In [10]:
# Fit the model for the lower layers
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.00001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

tbCallBack = keras.callbacks.TensorBoard(log_dir="../../LocalContent/Logs/08_KerasInceptionV3_TestingDirectionLoading/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers

history = model.fit_generator(
        train_generator,
        steps_per_epoch=N_training//batch_size,
        epochs=2,
        validation_data=validation_generator,
        validation_steps=N_verification//batch_size,
        max_queue_size=1,
        callbacks=[tbCallBack])


Epoch 1/2


ResourceExhaustedError: OOM when allocating tensor with shape[64,64,125,125] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: conv2d_3/convolution = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](activation_2/Relu, conv2d_3/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: metrics_4/acc/Mean/_5835 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_8436_metrics_4/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tens